# FuseTS - CropSAR
This Jupyter Notebook implements the CropSAR service, which is part of the FuseTS toolbox. The purpose of this notebook is to demonstrate the usage of openEO to execute the CropSAR service for a specific area of interest (AOI).

The CropSAR service is designed to provide valuable insights into crop monitoring and analysis by augmernting Sentinel-2 observations using that uses Sentinel-1 radar. This process enables the monitoring of agricultural fields regardless of weather conditions or daylight, making it a powerful tool for crop assessment.





## Setting up the OpenEO process
The first step includes setting up the OpenEO processing through the [OpenEO Python Client](https://open-eo.github.io/openeo-python-client/). Since the CropSAR algorithm is integrated as an [user defined process](https://open-eo.github.io/openeo-python-client/cookbook/udp_sharing.html), we can use the `datacube_from_process` function to execute the service.

In [1]:
import openeo
import datetime
import tempfile
import os
import xarray
import matplotlib.pyplot as plt

In [2]:
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
service = 'CropSAR_px'
namespace = 'vito'

Authenticated using refresh token.


In [3]:
connection.describe_process(service, namespace=namespace)

{'description': '# CropSAR_px\n\n## Description\n\nTechnical description of the algorithm.\n\n## Usage\n\nThis code example highlights the usage of the CropSAR_px process in an OpenEO batch job.\nThe result of this batch job will consist of individual GeoTIFF files per date.\nGenerating multiple GeoTIFF files as output is only possible in a batch job.\n\n```python\nimport openeo\n\n# define ROI and TOI\ngeometry = {\n    "type": "Polygon",\n    "coordinates": [\n        [\n            [\n                5.034656524658203,\n                51.20946446493662\n            ],\n            [\n                5.080232620239258,\n                51.20946446493662\n            ],\n            [\n                5.080232620239258,\n                51.234084900561015\n            ],\n            [\n                5.034656524658203,\n                51.234084900561015\n            ],\n            [\n                5.034656524658203,\n                51.20946446493662\n            ]\n        ]\n    ]\n}\n\nstartdate = "2020-05-01"\nenddate = "2020-06-01"\n\n# get datacube\nconnection = openeo.connect("https://openeo.vito.be")\ncube = connection.datacube_from_process(\n    "CropSAR_px",\n    namespace="vito",\n    geometry=geometry,\n    startdate=startdate,\n    enddate=enddate,\n    version=2,\n    model_path="tmp/model/cnn_transformer/",\n    path_extras=["tmp/env/env/"]\n)\njob = cube.execute_batch(\n    title="CropSAR_px",\n    out_format="GTIFF",\n    job_options={\n        "executor-cores": "8",\n        "task-cpus": "8",\n        "executor-memoryOverhead": "2g",\n        "udf-dependency-archives": [\n            "https://artifactory.vgt.vito.be/auxdata-public/cropsar_px/20230504T175919_cnn_transformer.zip#tmp/model/cnn_transformer",\n            "https://artifactory.vgt.vito.be/auxdata-public/cropsar_px/env.tar.gz#tmp/env"\n        ]\n    }\n)\nresults = job.get_results()\nresults.download_files("out")  # write files to output directory\n```\n\n### Scaling\nThe output contains NDVI values in the interval [-0.08; 1] scaled to the byte output range [0; 250].\nIf you want to convert back to the physical values, you have to use the following scaling factor: `x * 0.00432 - 0.08`\n\n## Limitations\n\nThe spatial extent is limited to a maximum size equal to a Sentinel-2 MGRS tile (100 km x 100 km).\nThe temporal extent is limited to 1 year.\n\nFor large spatial areas or temporal intervals, it is recommended to split the job in multiple smaller tasks.\n\n## Configuration & Resource Usage\n\nHere is an overview of run configurations for different ROI/TOI with memory requirements and estimated run durations.\n\n### Batch jobs\n\n| Spatial extent | Temporal extent | Executor memory | Run duration (sec) | Cost (credits) |\n|----------------|-----------------|-----------------|--------------------|----------------|\n| 100 m x 100 m  | 1 month         | default         | 171                | 22             |\n| 500 m x 100 m  | 1 month         | default         | 220                | 29             |\n| 1 km x 1 km    | 1 month         | default         | 220                | 29             |\n| 5 km x 5 km    | 1 month         | default         | 301                | 44             |\n| 10 km x 10 km  | 1 month         | 6 GB            | 520                | 90             |\n| 100m x 100 m   | 7 months        | default         | 258                | 32             |\n| 500 m x 500 m  | 7 months        | default         | 233                | 31             |\n| 1 km x 1 km    | 7 months        | default         | 248                | 35             |\n| 5 km x 5 km    | 7 months        | default         | 372                | 66             |\n| 10 km x 10 km  | 7 months        | 6 GB            | 736                | 133            |\n\nThe executor memory defaults to 5 GB. You can increase the executor memory by specifying it as a job option, eg:\n\n```python\njob = cube.execute_batch(out_format="GTIFF", job_options={"executor-memory": "7

connection.describe_process(service, namespace=namespace)## Retrieve `CropSAR_px` process
Look for `CropSAR_px` process in the `vito` namespace. Especially the *Parameters* section of the process is of interest.

### Use `datacube_from_process` to get initial DataCube
Get output datacube from process by passing in the process parameters:
**Mandatory**
- `geometry`: GeoJSON feature
- `startdate`: start of temporal interval
- `enddate`: end of temporal interval

**Optional**
- `nrt`: only use prior information (default: false)
- `inpaint_only`: only predict areas with no ground truth data (default: true)
- `output_mask`: output ground truth mask (deafult: false)
- `drop_dates`: list of dates

In [4]:
geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [
                5.034656524658203,
                51.20946446493662
            ],
            [
                5.080232620239258,
                51.20946446493662
            ],
            [
                5.080232620239258,
                51.234084900561015
            ],
            [
                5.034656524658203,
                51.234084900561015
            ],
            [
                5.034656524658203,
                51.20946446493662
            ]
        ]
    ]
}

dates = ['2021-07-01', '2021-07-31']

datacube = connection.datacube_from_process(
    service, namespace=namespace,
    geometry=geometry,
    startdate=dates[0],
    enddate=dates[1],
    version=2,
    model_path="tmp/model/cnn_transformer/",
    path_extras=["tmp/env/env/"]
)

/Users/bramjanssen/projects/vito/FuseTS/venv/lib/python3.8/site-packages/openeo/metadata.py:255: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


### Execute job and download result

In [5]:
output_file = './cropsar.nc'
job = datacube.execute_batch(
    title="FuseTS - CropSAR",
    out_format="netcdf",
    job_options={
        "executor-cores": "8",
        "task-cpus": "8",
        "executor-memoryOverhead": "2g",
        "udf-dependency-archives": [
            "https://artifactory.vgt.vito.be/auxdata-public/cropsar_px/20230504T175919_cnn_transformer.zip#tmp/model/cnn_transformer",
            "https://artifactory.vgt.vito.be/auxdata-public/cropsar_px/env.tar.gz#tmp/env"
        ]
    }
)
results = job.get_results().download_file('./cropsar.nc')

0:00:00 Job 'j-93cb50a9726e486c93ea4726f89017d0': send 'start'
0:02:41 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:02:46 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:02:53 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:03:01 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:03:11 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:03:23 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:04:05 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:04:24 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:04:48 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:05:18 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:05:56 Job 'j-93cb50a9726e486c93ea4726f89017d0': queued (progress N/A)
0:06:42 Job 'j-93cb50a9726e486c93ea4726f89017d0': running (progress N/A)
0:07:41 Job 'j-93cb50a9726e486c93ea4726f89017d0': running (progress N/A)

## Explore results
The first band of the output will be the CropSAR_px NDVI prediction.
The second band contains the Sentinel-2 input mask (if requested via `output_mask`). The values of this mask have the following meaning:
 - 0: no data (clouds, ...)
 - 1: ground truth data
 - 2: manually masked (via `drop_dates` parameter)

### Scaling
The output contains NDVI values in the interval [-0.08; 1] scaled to the byte output range [0; 250].
If you want to convert back to the physical values, you have to use the following scaling factor: `x * 0.00432 - 0.08`


In [ ]:
import holoviews as hv

output_file = './cropsar.nc'
ds = xarray.open_dataset(output_file)
ds = ds.where(ds != 255)  # filter no data value


hv.extension('bokeh')
%output holomap='scrubber'
%opts Image style(cmap='viridis') plot[colorbar=True]
%opts Image [width=500, height=400]
hv_ds = hv.Dataset(ds.NDVI)
hv_ds.to(hv.Image, ['x', 'y'])